In [ ]:
import numpy as np
def classic_fw(
    objective_fun, 
    gradient_fun, 
    LMO, 
    x0, 
    hyperparams = {"max_iterations": 20, "tolerance": 1e-6}
):
    """
    Inputs:
        - objective_fun (callable): Function f(x) to minimize (or maximize if sign=-1).
        - gradient_fun (callable): Gradient of f(x).
        - projection_operator (callable): Projection onto the feasible set (e.g., L∞-ball).
        - x0 (np.ndarray): Initial feasible point.
        - hyperparams (dict): Dictionary with hyperparameters:
            - "max_iterations" (int): Maximum number of iterations.
            - "tolerance" (float): Tolerance on gradient norm.
    Outputs:
        - x_t (np.ndarray): Final solution.
        - t (int): Number of iterations performed.
        - history (dict): Contains 'objective' and 'gradient_norm'.
    """
    # Defining the parameters
    x_t = x0.copy().astype(np.float64)
    max_iterations = hyperparams["max_iterations"]
    tolerance = hyperparams["tolerance"]

    # History trackers
    history = {'objective': [], 'gap': [], 'gradient': []}

    # Starting the Frank-Wolfe iterations
    for t in range(1, max_iterations + 1):
        
        # Compute the gradient at the current point
        grad_t = gradient_fun(x_t)

        # Compute the FW direction
        s_t = LMO(grad_t)
        d_t = s_t - x_t
        
        # Compute the duality gap
        gap = -grad_t @ d_t
        
        # Store history
        history["gradient"].append(grad_t)
        history["objective"].append(objective_fun(x_t))
        history["gap"].append(gap)

        # Check for convergence
        if gap < tolerance:
            print(f"Duality gap below tolerance at iteration {t}: {gap:.2e}")
            break

        # Compute the step size
        gamma_t = 2.0 / (t + 2.0)

        # Compute the next step
        x_t += gamma_t * d_t
        
    return x_t, t, history